<a href="https://colab.research.google.com/github/AbdullahFEliwa/DaimondPricePrediction/blob/main/Diamond_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
! mkdir ~/.kaggle/

In [ ]:
!cp /content/drive/MyDrive/projects/kaggle_1.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download shivam2503/diamonds

In [ ]:
!unzip diamonds.zip

**Data Exploration**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('diamonds.csv')
df

In [ ]:
df.info()

In [ ]:
df = df.drop(["Unnamed: 0"],axis=1)
df.head(5)

In [ ]:
#Remove Duplicate
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
df.describe()

In [ ]:
df = df.drop(df[df["x"]==0].index)
df = df.drop(df[df["y"]==0].index)
df = df.drop(df[df["z"]==0].index)
df.shape


**price**
price in US dollars (\$326--\$18,823)
**carat**
weight of the diamond (0.2--5.01)
**cut**
quality of the cut (Fair, Good, Very Good, Premium, Ideal)
**color**
diamond colour, from J (worst) to D (best)
**clarity**
a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
**x**
length in mm (0--10.74)
**y**
width in mm (0--58.9)
**z**
depth in mm (0--31.8)
**depth**
total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)
**table**
width of top of diamond relative to widest point (43--95)

In [ ]:
#depth =2*z/(x+y) Apply on first row in dataset

depth=2*2.43/(3.95+3.98)
depth

In [ ]:
df = df.drop(['x','y','z'],axis=1)
df.head(3)

In [ ]:
df.dtypes

In [ ]:
df["price"]=df["price"].astype(float)
df.dtypes

**DataVisualization and Outlier handle **

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 9))

# Plot first histogram in the first subplot
axs[0].hist(df["price"],bins=40)
axs[0].set_title('Distrbution of the price')

# Plot second histogram in the second subplot
axs[1].hist(df["carat"], bins=40)
axs[1].set_title('Distrbution of the carat')


In [ ]:
fig=plt.figure(figsize=(6,6))
HEAT=sns.heatmap(df.corr(),annot=True)
plt.show()

In [ ]:
df["price"].skew()

In [ ]:
num_feature=[feature for feature in df.columns if df[feature].dtype!='O']
print(num_feature)

In [ ]:
def outlier_handle(data, col_list):
    df_new = data.copy()
    for col_name in col_list:
        q1 = data[col_name].quantile(0.25)
        q3 = data[col_name].quantile(0.75)
        iqr = q3-q1  #IQR
        fence_low  = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        df_new.loc[:,  col_name] = np.where(data[col_name]> fence_high, fence_high,
                                            np.where(data[col_name]< fence_low,fence_low,data[col_name])
                                           )
    return df_new

In [ ]:
df=outlier_handle(df,num_feature)

In [ ]:
for col in num_feature:
    fig,ax=plt.subplots(1, 2, constrained_layout=True)
    fig.set_size_inches(15, 4)
    ax[0].hist(df[col], bins=50, density=True)
    ax[0].set_title('histoplot')

    ax[1].boxplot(df[col],vert=0)
    ax[1].set_title('Boxplot')

    #sns.distplot(df[col], ax=ax[0],color='blue').set(title="Distplot")
    #sns.boxplot(df[col], ax=ax[1],color='blue').set(title="Boxplot")
    plt.suptitle(f'{col.title()}',weight='bold')
    plt.show()

In [ ]:
df["price"].skew()

In [ ]:
import plotly.express as px
fig = px.scatter_3d(df, x='table', y='carat', z='price',color='clarity')
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter_3d(df, x='depth', y='carat', z='price',color='color')
fig.show()

In [ ]:
plt.subplots(figsize = (15, 8))
p = sns.violinplot(x = df["cut"],
                  y = df["price"],
                  order = df["cut"].value_counts().index,
                  saturation = 1,
                  linewidth = 4,
                  edgecolor = "black")

p.axes.set_title("\nDiamond's Prices on Cut\n", fontsize = 20)
p.axes.set_xlabel("\nCut", fontsize = 25)
p.axes.set_ylabel("Price", fontsize = 25)
sns.despine(left = True, bottom = True)
plt.show()

In [ ]:
plt.figure(figsize = (6, 4))

sns.countplot(y=df["cut"], order=df.cut.value_counts().index)

**Mapping For categrical parameter **

In [ ]:
#J (worst) to D (best)
df["color"].unique()

In [ ]:
df['color']=df.color.map({'J':1, 'I':2,  'H':3, 'G':4, 'F':5, 'E':6, 'D':7 })
df.head(2)

In [ ]:

df["cut"].unique()

In [ ]:
df['cut']=df.cut.map({'Fair':1, 'Good':2,  'Very Good':3, 'Premium':4, 'Ideal':5 })
df.head(2)

In [ ]:
#(I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
df["clarity"].unique()

In [ ]:
df['clarity']=df.clarity.map({'I1':1, 'SI2':2,  'SI1':3, 'VS2':4, 'VS1':5, 'VVS2':6, 'VVS1':7 , 'IF':8})
df.head(2)

In [ ]:
X= df.drop(["price"],axis =1)
y= df["price"]
X

**Split data **

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test =train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

In [ ]:
print(x_train_scaled.shape)
print(x_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

**Bulid model**

In [ ]:
from sklearn.linear_model import LinearRegression
Reg =LinearRegression()
Reg.fit(x_train_scaled,y_train)


In [ ]:
y_pred=Reg.predict(x_test_scaled)

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error

print("After Performing Linear Regression,\n")
print(f"R Squared Value: {metrics.r2_score(y_test, y_pred)}")

print(f"Adjusted R Squared Value: {1-(1-metrics.r2_score(y_test,y_pred))*(len(y_test)-1)/(len(y_test)-x_test_scaled.shape[1]-1)}")


print(f"Mean Absolute Error: {metrics.mean_absolute_error(y_test,y_pred)}")
print(f"Mean Squared Error: {metrics.mean_squared_error(y_test,y_pred)}")
print(f"Root Mean Squared Error: {metrics.mean_squared_error(y_test,y_pred,squared = False)}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

k = 10

skf = KFold(n_splits=k, shuffle=True, random_state=42)

data_splits_object=skf.split(X, y)

#Run the following 3 lines together multiple times to see how different data gets presented
my_split_data=next(data_splits_object)
print("Train indices are: ", my_split_data[0][0:10])
print("Test indices are: ", my_split_data[1][0:10]) #Print first 10 values

# empty lists to store predicted and ground truth values from all folds
predicted_y = []
expected_y = []


scaler = MinMaxScaler()

# Fit and predict iteratively on all splits
for train_index, test_index in skf.split(X, y):

    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Scale inputs
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    model = LinearRegression()
    model.fit(x_train, y_train)

    # print accuracy for each fold
    predicted_this_fold = model.predict(x_test)
    mse = mean_squared_error(y_test, predicted_this_fold)
    print("MSE from this fold is: " + str(mse))

    # store the prediction in a list
    predicted_y.extend(predicted_this_fold)

    # store the ground truth for this specific fold
    expected_y.extend(y_test)

# print accuracy
mse = mean_squared_error(expected_y, predicted_y)
print('\n', "MSE from all folds is: " + str(mse))